In [18]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [19]:
torch.cuda.set_device(0)

In [20]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [21]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [22]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [23]:
dimension = 2

In [24]:
# exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1)
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [25]:
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 0.25*pi**2*torch.sin(pi/2*(1-x_norm))+(dimension -1)*pi/(2*x_norm)*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [26]:
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2    
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + u_hat**3 - f_temp)**2)/x.size()[0]
    
    Nb = 200
    theta = 2*pi*(2*torch.rand(Nb)-1.0)
    y = torch.rand(Nb, dimension).cuda()
    y[:, 0] = torch.cos(theta)
    y[:, 1] = torch.sin(theta)
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 50.0
    return part_1 + lambda1 * part_2 

In [27]:
Data_size = 2000
def Gendata():
    i=0
    notes = torch.tensor([])
    while notes.size()[0]< Data_size:
        temp_notes = 2*torch.rand(1000,dimension)-1
        index=torch.where(torch.norm(temp_notes,dim=1)<1)
        temp_notes = temp_notes[index]
        notes = torch.cat((notes,temp_notes),0)
    notes = notes[1:Data_size,:]
    return notes.cuda()

In [28]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [29]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [30]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=2, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [31]:
import torch.optim as optim
import torch.nn as nn
import time

In [32]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [33]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [34]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_2D_Dirichlet_nonlinear_lambda50.0.npy", error_save)
np.save("DGM_relative_error_2D_Dirichlet_nonlinear_lambda50.0.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(24.1224, device='cuda:0')
current relative error is:  tensor(1.3862, device='cuda:0')
current epoch is:  50
current loss is:  tensor(4.7078, device='cuda:0')
current relative error is:  tensor(0.5503, device='cuda:0')
current epoch is:  100
current loss is:  tensor(1.4111, device='cuda:0')
current relative error is:  tensor(0.1691, device='cuda:0')
current epoch is:  150
current loss is:  tensor(0.5425, device='cuda:0')
current relative error is:  tensor(0.0777, device='cuda:0')
current epoch is:  200
current loss is:  tensor(0.2171, device='cuda:0')
current relative error is:  tensor(0.0410, device='cuda:0')
current epoch is:  250
current loss is:  tensor(0.1325, device='cuda:0')
current relative error is:  tensor(0.0272, device='cuda:0')
current epoch is:  300
current loss is:  tensor(0.0910, device='cuda:0')
current relative error is:  tensor(0.0227, device='cuda:0')
current epoch is:  350
current loss is:  tensor(0.0646, device='cuda:0'

current epoch is:  3100
current loss is:  tensor(0.0009, device='cuda:0')
current relative error is:  tensor(0.0012, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(0.0009, device='cuda:0')
current relative error is:  tensor(0.0012, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(0.0009, device='cuda:0')
current relative error is:  tensor(0.0012, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(0.0009, device='cuda:0')
current relative error is:  tensor(0.0013, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(0.0009, device='cuda:0')
current relative error is:  tensor(0.0018, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(0.0008, device='cuda:0')
current relative error is:  tensor(0.0010, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(0.0008, device='cuda:0')
current relative error is:  tensor(0.0010, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(0.0008, devic

current epoch is:  6200
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0013, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0011, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0013, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(0.0004, device='cuda:0')
current relative error is:  tensor(0.0011, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0026, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0013, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(0.0011, device='cuda:0')
current relative error is:  tensor(0.0028, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(0.0004, devic

current epoch is:  9300
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0010, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.0002, device='cuda:0')
current relative error is:  tensor(0.0019, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0013, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.0002, device='cuda:0')
current relative error is:  tensor(0.0015, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.0004, device='cuda:0')
current relative error is:  tensor(0.0031, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0009, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.0002, device='cuda:0')
current relative error is:  tensor(0.0035, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.0002, devic

current epoch is:  12350
current loss is:  tensor(7.5083e-05, device='cuda:0')
current relative error is:  tensor(0.0007, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0014, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(7.1042e-05, device='cuda:0')
current relative error is:  tensor(0.0007, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.0002, device='cuda:0')
current relative error is:  tensor(0.0009, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0027, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0006, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.0002, device='cuda:0')
current relative error is:  tensor(0.0047, device='cuda:0')
current epoch is:  12700
current loss is:  tens

current epoch is:  15350
current loss is:  tensor(5.3074e-05, device='cuda:0')
current relative error is:  tensor(0.0006, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(9.4277e-05, device='cuda:0')
current relative error is:  tensor(0.0014, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0052, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0018, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.0003, device='cuda:0')
current relative error is:  tensor(0.0027, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.0021, device='cuda:0')
current relative error is:  tensor(0.0130, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.0004, device='cuda:0')
current relative error is:  tensor(0.0031, device='cuda:0')
current epoch is:  15700
current loss is:  tens

current epoch is:  18350
current loss is:  tensor(4.2676e-05, device='cuda:0')
current relative error is:  tensor(0.0005, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(4.3014e-05, device='cuda:0')
current relative error is:  tensor(0.0006, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(0.0001, device='cuda:0')
current relative error is:  tensor(0.0016, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(5.4137e-05, device='cuda:0')
current relative error is:  tensor(0.0007, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(5.1782e-05, device='cuda:0')
current relative error is:  tensor(0.0007, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(6.7440e-05, device='cuda:0')
current relative error is:  tensor(0.0011, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(5.5226e-05, device='cuda:0')
current relative error is:  tensor(0.0006, device='cuda:0')
current epoch is:  18700
curren